In [4]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [5]:
# CHANGE ME!
TASK_ID = 'purchased_all_food'
AUTHOR = 'will'

In [43]:
task = Task(
    id=TASK_ID,
    category="First-order logic",
    name="People that purchased all possible items",
    description="Find the name of all buyers that ordered every food item at least once.",
    plan=[
    ],
    sample_input={
        "food": [
            {"id": 1, "name": "Burrito"},
            {"id": 2, "name": "Sushi"}
        ],
        "orders": [
            {"id": 1, "buyer": "Will", "food": 1},
            {"id": 2, "buyer": "Scott", "food": 2},
            {"id": 3, "buyer": "Scott", "food": 2},
            {"id": 4, "buyer": "Will", "food": 2},
        ]
    },
    sample_output=["Will"],
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [14]:
sql = replace(prototype,
    language='sql',
    source='''SELECT DISTINCT buyer
FROM orders o1
WHERE 
  (SELECT COUNT(DISTINCT food) FROM orders o2 WHERE o1.buyer = o2.buyer) = 
  (SELECT COUNT(*) FROM food)
''').load_plan()
sql.execute(task)
sql.save()

In [20]:
datalog = replace(prototype,
    language='datalog',
    source='''
.decl has_purchased(buyer:symbol, food:number)
has_purchased(buyer, food) :- orders(buyer, food, _).

purchased_all_food(buyer) :-
  orders(buyer, _, _),
  n_food = count : food(_, _),
  n_unique_orders = count : has_purchased(buyer, _),
  n_food = n_unique_orders.''').load_plan()
datalog.execute(task)
datalog.save()

In [37]:
pandas = replace(prototype,
    language='python-pandas',
    source='''def purchased_all_food(food, orders):
    n_food = len(food)
    n_unique_orders = (orders
        .groupby('buyer')
        .food.unique() 
        .map(len))
    return n_unique_orders[n_unique_orders == n_food].index.values''').load_plan()
pandas.execute(task)
pandas.save()

In [39]:
imperative = replace(prototype,
    language='python-imperative',
    source='''def purchased_all_food(food, orders):
    n_food = len(food)
    unique_orders = defaultdict(set)
    for order in orders:
      unique_orders[order["buyer"]].add(order["food"])
      
    buyers = []
    for buyer, orders in unique_orders.items():
      if len(orders) == n_food:
        buyers.append(buyer)
    return buyers
    ''').load_plan()
imperative.execute(task)
imperative.save()

In [40]:
functional = replace(prototype,
    language='python-functional',
    source='''def purchased_all_food(food, orders):
    n_food = len(food)
    buyers = set([order["buyer"] for order in orders])
    return [
        buyer
        for buyer in buyers
        if len(set([order["food"] for order in orders 
             if order["buyer"] == buyer])) == n_food
    ]''').load_plan()
functional.execute(task)
functional.save()